In [ ]:
# ChessCompressConcurrently, the Kijk 1985 chess challenge revisited and made concurrent, for current times' sake

In [ ]:
# set this notebook to use a large part of the browser window width
from IPython.core.display import HTML, display
display(HTML("<style>.container { width:80% !important; }</style>"))

# Leveraging our Previous Notebook

## Introduction

### Previous Chess Compress Challenge from 1985

This notebook expands on the ChessCompress.ipynb one. In that notebook we solved the problem of trying to put as many chess pieces on a board without one threatening another.
More precisely, we want to maximise the score of the solution of a problem with the mentioned constraints, but where the score is calculated as follows.
Each queen and rook get 1/8 points. Each bishop gets 1/14 points. Each king gets 1/16 points and a knight gets 1.32 points. Paws are not allowed in the game.

### New Multi-Floor  Chess Compress (Concurrent) Challenge from 2020

We present a new take on this problem here, where only the non-threatening constraints are to be respected between pieces of the same family,
but pieces of a different family are supposed to not threaten each other, say because they would 'live' on different floors.

Code was developed in that other notebook ChessCompress.ipynb, but is repeated here so that it is self-contained and we can independently execute it. 
So if you want to have more written text to understand the need or rationale for this code refer to this ChessCompress.ipynb notebook.

### Code developed in the ChessCompress.ipynb Notebook

In [ ]:
%%sh
pip3 install python-chess

In [ ]:
import chess
board = chess.Board()
board

In [ ]:
def write_board_to_svg(board, filename):
    import chess.svg
    boardsvg = chess.svg.board(board)
    f = open(filename, "w")
    f.write(boardsvg)
    f.close()
write_board_to_svg(board, 'OpeningChessBoard.svg')

In [ ]:
board = chess.Board("rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR")
write_board_to_svg(board, 'FirstMoveBoard.svg')
board

In [ ]:
from gurobipy import *
import itertools

In [ ]:
class ChessCompress():
    
    def __init__(self, dim, piece_to_weight_dict):
        self.dim = dim
        self.m = Model()
        self.board_vars = {}  # dictionary from piece type letter 
        # to (row, col) tuple to binary solver var. 
        self.board_vals = {}  # dictionary from piece type letter 
        # to (row, col) tuple to binary result val.
        self.count = {}  # dictionary from piece type letter 
        # to number of piece fittable
        self.add_variables(piece_to_weight_dict)
        
    def derive_parameters(self, dim):
        n_rows = dim
        n_cols = dim
        rows = list(range(n_rows))
        cols = list(range(n_cols))
        return n_rows, n_cols, rows, cols

    # define the binary core variables
    def add_variables(self, piece_to_weight_dict):
        n_rows, n_cols, rows, cols = self.derive_parameters(self.dim)
        for piece_letter in piece_to_weight_dict:
            piece_weight = piece_to_weight_dict[piece_letter]
            pieces = {(r, c): -piece_weight \
                      for r, c in itertools.product(rows, cols)}
            self.board_vars[piece_letter] = \
                self.m.addVars(pieces.keys(), obj=pieces, 
                               vtype=GRB.BINARY, name=piece_letter)
    
    def add_long_range_piece_constraints(
        self, from_piece_letter, to_piece_letter, 
        piece_weight=1.0, user_cuts=True):
        
        verbose = 0
        
        from_piece_upper_letter = from_piece_letter.upper()
        to_piece_upper_letter = to_piece_letter.upper()

        n_rows, n_cols, rows, cols = self.derive_parameters(self.dim)

        # These constraints are sufficient to define 
        # the allowed search space.
        vertical_horizontal_dr_list = [ -1,  0, +1,  0] 
        vertical_horizontal_dc_list = [  0, +1,  0, -1]

        diagonal_dr_list = [ -1, +1, +1, -1]
        diagonal_dc_list = [ +1, +1, -1, -1]

        dr_list = []
        dc_list = []
        if from_piece_letter in ['q', 'r']:
            dr_list.extend(vertical_horizontal_dr_list)
            dc_list.extend(vertical_horizontal_dc_list)
        if from_piece_letter in ['q', 'b']:
            dr_list.extend(diagonal_dr_list) 
            dc_list.extend(diagonal_dc_list)
        if verbose > 0:
            print(dr_list)
            print(dc_list)
        
        for r in rows:
            for c in cols:
                # dr = delta row, cr is delta column
                for dr, dc in list(zip(dr_list, dc_list)):
                    r_, c_ = r+dr, c+dc
                    # while (d+dr, c+dc) square falls within board
                    while (r_ <= self.dim-1 and r_ >= 0) \
                        and (c_ <= self.dim-1 and c_ >= 0):  
                        constr_name = \
                            'noQBThreat_r{:d}_c{:d}_dr{:d}_dc{:d}'.\
                            format(r, c, dr, dc)
                        self.m.addConstr(\
                            self.board_vars[from_piece_letter][r,c] +\
                            self.board_vars[to_piece_letter][r_,c_] <= 1, 
                            constr_name)
                        r_, c_ = r_+dr, c_+dc
            
        # The next constraints can be optionally added.
        # They are 'user cuts' intended to speed up the solving process.
        #They do not restrict the search
        # space compared to the constraint already defined above.
        # The above constraint work on a pair of binary variables,
        # while the below 'user cut' ones work for all binary variables
        # in a full row, column or diagonal at once.
        if (from_piece_letter == to_piece_letter) and user_cuts:
            # row-wise constraints
            if from_piece_letter in ['q', 'r']:
                for r in rows:
                    constr_name = \
                        'atMostOne' + from_piece_upper_letter +\
                        'InRow_r{:d}'.format(r)
                    self.m.addConstr(
                        quicksum(self.board_vars[from_piece_letter][r,c] \
                                 for c in cols) <= 1, 
                                constr_name)

                # column-wise constraints
                for c in cols:
                    constr_name = 'atMostOne' + \
                    from_piece_upper_letter + \
                    'InCol_c{:d}'.format(c)
                    self.m.addConstr(
                        quicksum(self.board_vars[from_piece_letter][r,c] \
                                 for r in rows) <= 1, 
                                     constr_name)

            # all diagonal constraints:
            if from_piece_letter in ['q', 'b']:
                for dr, dc in [(-1, +1), (+1, +1)]:
                    if (dr, dc) == (-1, +1):
                        # go over diagonals that go RightUp
                        # start (r,c) for these diagonals are 
                        # left border of board and 
                        start_r_list = rows[1:]
                        start_c_list = [0] * (self.dim-1)
                        # lower border of board
                        start_r_list.extend([self.dim-1] * (self.dim-2))
                        start_c_list.extend(cols[1:][:-1])
                        constr_name = 'atMostOne' + \
                            from_piece_upper_letter + 'InRightUpDiag'
                    else:
                        # go over diagonals that go RightD(ow)n
                        # start (r,c) for these diagonals are 
                        # top border of board and
                        start_r_list = [0] * (self.dim-1)
                        start_c_list = cols[:-1]  # last col dropped 
                        # since would result in a 1 q var 
                        # constraint <=1 which is redundant.
                        # left border of board
                        start_r_list.extend(rows[1:][:-1])
                        start_c_list.extend([0] * (self.dim-2))
                        constr_name = 'atMostOne' + \
                            from_piece_upper_letter + 'InRightDnDiag'

                    if verbose > 0:
                        print(constr_name + ':')
                        print('  for start_r range: [' + ','.\
                              join([str(i) for i in start_r_list]) + ']')
                        print('  for start_c range: [' + ','.\
                              join([str(i) for i in start_c_list]) + ']')
                    assert len(start_r_list) == len(start_c_list)
                    for start_r, start_c in list(zip(start_r_list, 
                                                     start_c_list)): 
                        r, c = start_r, start_c
                        expr = LinExpr(0)
                        while (c >= 0 and c <= self.dim-1) and\
                        (r >= 0 and r <= self.dim-1):
                            if verbose > 0:
                                print(r,c)
                            expr += self.board_vars[from_piece_letter][r,c]
                            r += dr
                            c += dc
                        if verbose > 0:
                            print('---')
                        self.m.addConstr(expr <= 1, 
                                         constr_name + \
                                         '_rs{:d}cs{:d}'.format(r,c))

    def set_solver_output(self, bool):
        self.m.setParam('OutputFlag', bool)
        
    def set_time_limit(self, time_limit_in_seconds):
        self.m.setParam('TimeLimit', time_limit_in_seconds)             
            
    def solve_model(self):
        # minimise the objective function the model 
        # Since coefficients in the objective are negative this will try to 
        # maximize the number of queens in the board.
        self.m.optimize()

        # retrieve solution
        for piece_letter in self.board_vars:
            self.board_vals[piece_letter] = \
                self.m.getAttr('x', self.board_vars[piece_letter])
            # the board_vals represent the board solutions 
            # without any solver specific encoding
            self.count[piece_letter] = 0
            for r,c in self.board_vals[piece_letter]:
                if self.board_vals[piece_letter][r,c]==1:
                    self.count[piece_letter] += 1            
                
        n_different_piece_letters = len(self.board_vars.keys())        
        if n_different_piece_letters == 1:
            assert self.count[piece_letter] == \
                -int(self.m.getObjective().getValue())
        
    def get_objective_value(self):
        return self.m.getObjective().getValue()

    def get_mip_gap(self):
        return self.m.getAttr('MIPGap')
            
    def get_letter(self, r, c):
        vals = self.board_vals
        found_letter = ' ' # means nothing found yet, so empty square
        for letter in vals:
            val = int(vals[letter][(r,c)])
            if val == 1:
                assert found_letter == ' '  # can only find one letter thanks to 
                # add_at_most_a_piece_per_square_constraints being called before
                found_letter = letter
                # keep checking that no other letter is found as well
        return found_letter
        
    def encode_to_fen_board_string(self, capitals_for_white=False):
        vals = self.board_vals
        n_rows, n_cols, rows, cols = self.derive_parameters(self.dim)
        s = ''
        space = ' '
        for r in rows:
            for c in cols:  # we assume these couples  lexically sorted
                letter = self.get_letter(r,c)
                bw_letter = letter.upper() if capitals_for_white else letter
                if self.dim == 8:  # then we keep to the standard fen format 
                    # since we can use chess.Board(on it) later
                    if letter != ' ':
                        s += bw_letter
                    else:
                        assert letter == ' '
                        if (len(s) == 0):
                            s = '1'
                        elif s[-1] in ['1','2','3','4','5','6','7','8','9']:
                            s = s[:-1] + str(int(s[-1])+1)
                        else:  # increment last digit of s
                            s += '1'

                    if c == self.dim-1:
                        if r != self.dim-1:
                            s += '/'
                else: # The additional 'space' here is 
                    # there to end up with square boards
                    if letter != ' ':
                        s += bw_letter + space
                    else:
                        assert letter == ' '
                        s += '.' + space
                        
                    if c == self.dim-1:
                        if r != self.dim-1:
                            s += '\n'

        return s

## Queens only Problem

In [ ]:
cc_queens = ChessCompress(8, {'q' : 1.0})
cc_queens.add_long_range_piece_constraints('q', 'q')
cc_queens.set_solver_output(True)
cc_queens.solve_model()
print('I could maximally fit {:d} queens on the chess board.'.\
      format(cc_queens.count['q']))
n_queens = cc_queens.count['q']

board_string = \
    cc_queens.encode_to_fen_board_string(capitals_for_white=True)
print(board_string)
board8Q = chess.Board(board_string)
write_board_to_svg(board8Q, 'EightQueensBoard.svg')
board8Q

## Rooks only Problem

In [ ]:
cc_rooks = ChessCompress(8, {'r':1.0})
cc_rooks.add_long_range_piece_constraints('r', 'r')
cc_rooks.m.setParam('OutputFlag', False)
cc_rooks.solve_model()
print('I could maximally fit {:d} rooks on the chess board.'.\
      format(cc_rooks.count['r']))
n_rooks = cc_rooks.count['r']

board_string = \
    cc_rooks.encode_to_fen_board_string(capitals_for_white=True)
print(board_string)
board8R = chess.Board(board_string)

write_board_to_svg(board8R, 'EightRooksBoard.svg')
board8R

## Bishops only Problem

In [ ]:
cc_bishops = ChessCompress(8, {'b': 1.0})
cc_bishops.add_long_range_piece_constraints('b','b')
cc_bishops.m.setParam('OutputFlag', False)
cc_bishops.solve_model()
print('I could maximally fit {:d} bishops on the chess board.'.\
      format(cc_bishops.count['b']))
n_bishops = cc_bishops.count['b']

board_string = \
    cc_bishops.encode_to_fen_board_string(\
        capitals_for_white=True)
print(board_string)
board16B = chess.Board(board_string)

write_board_to_svg(board16B, 'SixteenBishopsBoard.svg')
board16B

## Kings only Problem

In [ ]:
class ChessCompress(ChessCompress):  # extend the class
    
    def add_kings_constraints(self, to_piece_letter, 
                              piece_weight=1.0):
        n_rows, n_cols, rows, cols = self.derive_parameters(self.dim)
        kings = {(r, c): -piece_weight for r, 
                 c in itertools.product(rows, cols)}

        n_rows, n_cols, rows, cols = \
            self.derive_parameters(self.dim)
        kings = {(r, c): -piece_weight for r, 
                 c in itertools.product(rows, cols)}

        for r in rows:
            for c in cols:
                # dr = delta row, cr is delta column
                dr_list = [ -1,  0, +1, +1, +1, 0, -1, -1]
                dc_list = [ +1, +1, +1,  0, -1, -1, -1, 0]
                for dr, dc in list(zip(dr_list, dc_list)):
                    if (r+dr <= self.dim-1 and r+dr >= 0) \
                        and (c+dc <= self.dim-1 and c+dc >= 0):  
                        # if (d+dr, c+dc) square falls within board
                        constr_name = \
                            'noKingsThreat_r{:d}_c{:d}_dr{:d}_dc{:d}'.\
                        format(r, c, dr, dc)
                        self.m.addConstr(
                            self.board_vars['k'][r,c] + 
                            self.board_vars[to_piece_letter][r+dr,c+dc] <= 1, 
                            constr_name)

cc_kings = ChessCompress(8, {'k':1.0})
cc_kings.add_kings_constraints('k')
cc_kings.m.setParam('OutputFlag', False)
cc_kings.solve_model()
print('I could maximally fit {:d} kings on the chess board.'.
      format(cc_kings.count['k']))
board_string = \
    cc_kings.encode_to_fen_board_string(capitals_for_white=True)
n_kings = cc_kings.count['k']

print(board_string)
board16K = chess.Board(board_string)

write_board_to_svg(board16K, 'SixteenKingsBoard.svg')
board16K

## Knights only Problem

In [ ]:
class ChessCompress(ChessCompress):  # extend the class
    
    def add_knights_constraints(self, to_piece_letter, piece_weight=1.0):
        verbose = 0

        n_rows, n_cols, rows, cols = self.derive_parameters(self.dim)
        kings = {(r, c): -piece_weight for r, c in itertools.product(rows, cols)}

        # define the basic Constraints

        # row-wise constraints
        for r in rows:
            for c in cols:
                # dr = delta row, cr is delta column
                dr_list = [-2, -1, +1, +2, +2, +1, -1, -2]
                dc_list = [+1, +2, +2, +1, -1, -2, -2, -1]
                for dr, dc in list(zip(dr_list, dc_list)):
                    if (r+dr <= self.dim-1 and r+dr >= 0) \
                        and (c+dc <= self.dim-1 and c+dc >= 0):  
                        # if (d+dr, c+dc) square falls within board
                        constr_name = \
                            'noTwoNWithinSingleJump_r{:d}_c{:d}_dr{:d}_dc{:d}'.\
                            format(r, c, dr, dc)
                        self.m.addConstr(
                            self.board_vars['n'][r,c] + 
                            self.board_vars[to_piece_letter][r+dr,c+dc] <= 1, 
                            constr_name)

cc_knights = ChessCompress(8, {'n':1.0})
cc_knights.add_knights_constraints('n')
cc_knights.m.setParam('OutputFlag', False)
cc_knights.solve_model()
print('I could maximally fit {:d} knights on the chess board.'.
      format(cc_knights.count['n']))
print('The board setup scores {:f}.'.format(-cc_knights.get_objective_value()))

board_string = cc_knights.encode_to_fen_board_string(capitals_for_white=True)
n_knights = cc_knights.count['n']
print(board_string)
board32N = chess.Board(board_string)

write_board_to_svg(board32N, 'ThirtyTwoKnightsBoard.svg')
board32N

So 32 knights is the maximum fittable on a board, so they get the weight of 1/32 in the final problem. They are on (all) the squares of one color only, white here, but another solution would be for them on all the black squares. 

Let's summarize by writing a function that calculates all the weights for a given board dimension.

In [ ]:
def calculate_piece_weights(dim, user_cuts=True, 
                            capitals_for_white=True, solver_output=False):
    weights = {}
    boards = {}
    
    cc = ChessCompress(dim, {'k':1.0})
    cc.add_kings_constraints('k')
    cc.set_solver_output(solver_output)
    cc.solve_model()
    
    weights['k'] = 1/(-cc.get_objective_value())
    s = cc.encode_to_fen_board_string(capitals_for_white)
    boards['k'] = chess.Board(s) if dim == 8 else HTML(s)
    for piece_letter in ['q', 'b', 'r']:
        cc = ChessCompress(dim, {piece_letter:1.0})
        cc.add_long_range_piece_constraints(piece_letter, piece_letter, 
                                            {piece_letter:1.0}, user_cuts)
        cc.set_solver_output(solver_output)
        cc.solve_model()
        weights[piece_letter] = 1/(-cc.get_objective_value())
        s = cc.encode_to_fen_board_string(capitals_for_white)
        boards[piece_letter] = chess.Board(s) if dim == 8 else HTML(s)

    cc = ChessCompress(dim, {'n':1.0})
    cc.add_knights_constraints('n')
    cc.set_solver_output(solver_output)
    cc.solve_model()
    weights['n'] = 1/(-cc.get_objective_value())
    s = cc.encode_to_fen_board_string(capitals_for_white)
    boards['n'] = chess.Board(s) if dim == 8 else HTML(s)
     
    return weights, boards

weights_dim8, boards = calculate_piece_weights(8, user_cuts=True, 
                                               capitals_for_white=False)
for key, val in weights_dim8.items():
    print(key, int(1/val))
print(weights_dim8)

# The 2020 Problem, a Manual Solving Attempt

If you think it's fun to first try and aim to come up with some as good as possible solutions (optimal or not) without algorithms, 
here we go. Combining the horses only solution with the bishops only solution, we get this solution.

In [ ]:
fen = "NBN1N1N1/1N1N1N1N/N1N1N1NB/1N1N1N1N/N1N1N1NB/1N1N1N1N/N1N1N1N1/BN1N1NBN"
board = chess.Board(fen)
write_board_to_svg(board, 'ChessCompressConcurrentPeter1Board.svg')
board

It's a bit exhausting, error prone and boring for a human to calculate the score, but I imagine that for a computer, it must be fun. :)
So let's write a function to do that based on the fen string and the piece weights as the two function inputs.

In [ ]:
def calculate_score(fen, weights_dim8):
    score = 0.0
    for c in fen:
        c = c.lower()
        if c in weights_dim8.keys():
            score += weights_dim8[c]
        
    return score

calculate_score(fen, weights_dim8)

# The 2020 Problem, Solved to Optimality by Integer Programming

## Teams, assume position, all Piece Types get Equal Score 1

Hmm, shall we try to piece things together? 
Surely we still have to add a constraint saying that any square can contain at most one chess piece.

### Plain Vanilla Model, no user cuts

In [ ]:
class ChessCompress(ChessCompress):  # extend the class

    def add_at_most_a_piece_per_square_constraints(self):
        n_rows, n_cols, rows, cols = self.derive_parameters(self.dim)
        for r in rows:
            for c in cols:
                constr_name = \
                'atMostOnePiecePerSquare_r{:d}_c{:d}'.format(r, c)
                self.m.addConstr(
                    quicksum(self.board_vars[letter][r,c] 
                             for letter in self.board_vars) <= 1, 
                    constr_name)

In [ ]:
def max_pieces_on_board_forbidding_threat_within_own_type_only(
    piece_weights, user_cuts=False, solver_output=False):
    
    cc_all = ChessCompress(8, piece_weights)
    for piece_letter in ['q', 'b', 'r']:
        cc_all.add_long_range_piece_constraints(
            piece_letter, piece_letter,
            piece_weights[piece_letter], user_cuts)
    cc_all.add_kings_constraints('k')
    cc_all.add_knights_constraints('n')
    cc_all.add_at_most_a_piece_per_square_constraints()
    cc_all.set_solver_output(solver_output)
    cc_all.solve_model()
    solver_time_in_seconds = cc_all.m.getAttr('Runtime')
    print('Needed {:f} seconds to find optimal solution.'.format(
        solver_time_in_seconds))
    print('I could maximally fit {:d} queens, {:d} kings, {:d} bishops,'.
          format(cc_all.count['q'], cc_all.count['k'], cc_all.count['b']))
    print('  {:d} knights and {:d} rooks together on the chess board.'.
          format(cc_all.count['n'], cc_all.count['r']))

    print('The board setup scores {:f}.'.format(-cc_all.get_objective_value()))

    board_string = cc_all.encode_to_fen_board_string(capitals_for_white=True)
    print(board_string)
    board = chess.Board(board_string)
    return board

# For now we assume here that each piece gets a score of 1.
max_pieces_board = max_pieces_on_board_forbidding_threat_within_own_type_only(
    piece_weights={'q':1, 'k':1, 'b':1, 'r':1, 'n':1}, 
    user_cuts=False)


write_board_to_svg(max_pieces_board, 'MaxPiecesBoard.svg')
max_pieces_board

Now that's surprising: 63 pieces could be fit but nothing in the last square. Verified with our score calculating function, the score of this solution is indeed 63.

In [ ]:
equal_one_weights = {'k': 1.0, 'q': 1.0, 'b': 1.0, 'r': 1.0, 'n': 1.0}
calculate_score("NKBKNKNR/RNBNQNBB/NQNKNKNK/KBRN1QBN/NBNRNKNK/KBKNRBQN/NRNQNBNK/KNKNKNRN", equal_one_weights)

### Improving Time to Optimal Solution by Adding  User Cuts

The above problem takes 88 seconds to solve on my laptop. Maybe this time can be improved with adding the above mentioned user cuts. Let's try.

In [ ]:
max_pieces_user_cuts_board = max_pieces_on_board_forbidding_threat_within_own_type_only(
    piece_weights={'q':1, 'k':1, 'b':1, 'r':1, 'n':1}, 
    user_cuts=True)

write_board_to_svg(max_pieces_user_cuts_board, 'MaxPiecesUserCutsBoard.svg')
max_pieces_user_cuts_board

Yes! We only need about 7 seconds when including the user cuts. So this shows how big an effect this can have. In fact user cuts in business questions can come from business logic. It is something the solver cannot automatically identify, but the human modeler can add by having some higher level insight.

The solution has of course the same 'quality', objective function value -63 as the previous one without user cuts, but it need not be the same, and indeed here it isn't. Such is the nature of user cuts.
Let's double check our score just to be sure.

In [ ]:
calculate_score("KNKNKNRN/NRNBNQNK/KNKNKNBQ/NQN1NRNK/KNKNRNQN/BBQRNKNK/KNRBBNBN/NNNKNKBR", equal_one_weights)

Okay, that's correct, since the score should equal the number of pieces on the board here.

## Adding the Correct Weights per Chess Piece

For now, we have weighted pieces with the same weight 1. Maybe if we weigh them differently, we get a different solution or even structure. Let's try.

### Plain Vanilla Model, Without User Cuts

Let's try again first without user cuts, to see how long it takes Gurobi.

In [ ]:
max_weighted_pieces_board = max_pieces_on_board_forbidding_threat_within_own_type_only(
    piece_weights=weights_dim8, 
    user_cuts=False)

write_board_to_svg(max_weighted_pieces_board, 'MaxWeightedPiecesBoard.svg')
max_weighted_pieces_board

Ha, by introduction of these different weights we get that the optimal solution does not contain 63 pieces anymore but only 61. The solver cannot improve the solution by putting any piece on the leftover 3 squares. Double checking with weights all equal to 1 gives 

In [ ]:
optimal_2020_fen = "KBBQKBRB/N1KRNQNK/KN1NRKBQ/BQNKNRNK/BKRN1NQB/QRNKNKNK/BNQNBNBR/RKBKQKBK"
calculate_score(optimal_2020_fen, equal_one_weights)

indeed.


Double checking the score, but now with the different weights, gives

In [ ]:
calculate_score(optimal_2020_fen, weights_dim8)

indeed.

### Improving Time to Optimal Solution by Adding  User Cuts

Let's try again with user cuts, to see if this speeds up Gurobi, also in this weighted case.

In [ ]:
max_weighted_pieces_User_cuts_board = max_pieces_on_board_forbidding_threat_within_own_type_only(
    piece_weights=weights_dim8, 
    user_cuts=True)

write_board_to_svg(max_weighted_pieces_User_cuts_board, 'MaxWeightedPiecesUserCutsBoard.svg')
max_weighted_pieces_User_cuts_board

So this is again a solution with 'the same' 63 pieces but not the same positioning as found without user cuts.

In [ ]:
optimal_2020_fen = "KNKBBQBR/B1NQKRNK/BK1NRNQB/QRNKNKNK/KNRN1NBQ/BQKRNKNK/RNBNQNBB/BKQKBKRK"
calculate_score(optimal_2020_fen, equal_one_weights)

and with the correct different weights

In [ ]:
calculate_score(optimal_2020_fen, weights_dim8)

So this is an optimal solution to the 2020 ChessCompressConcurrent challenge.
The top score, given by this solution found by Gurobi is $4.428571$. 
Gurobi says: "No other solutions better than -4.428571" and that means it has proven this solution is optimal. 
To be totally exact about the final number, we can express it in rational number notation.

The $8$ queens give score $8/8=1$, 16 kings give $16/16=1$, 13 bishops give $13/14$, 16 knights give $16/32=1/2$ and 8 rooks give $8/8=1$
together that makes 1 + 1 + (1-1/14) + 1/2 + 1 = 5.5 - 1/14 = 9/2 -1/14 = (9*7 - 1)/14 = 62/14.

So this takes about 19 seconds on my laptop with the same user cuts, compared to about 53 seconds without user cuts.
A significant improvement again.

### Analysing the Structure of the Optimal Solution

There are some remarkable properties to be found in the optimal solution.

First, there are many piece types that score a full one (queens, kings and rooks). So these make up a combined score of 3.
Then the combined score of the knights is 16/32, so we can only put half of the number of knight (32) compared to the case where no other pieces would be allowed.
For the bishops we just have 13 i.o. 14.
So are the sub-structures formed per type of piece quite similar to the structures of the optimal solution to the problems of the solution with one allowed piece type only?
We guess they must be. But a human cannot easily separate the combined solution into the per piece type sub-solutions.
A computer can (when taught).

In [ ]:
def get_subsolution_fen_and_subscore(fen, weights_dim8, piece_type):
    terminated_fen = fen + '/'
    digits = [str(x) for x in list(range(0,10))]  # from 0 to 9
    sub_fen = ''
    sub_score = 0.0
    accumulated_read_not_yet_written_pieces_in_current_row = 0

    assert piece_type in weights_dim8.keys()                        
    unit_piece_type_score = weights_dim8[piece_type]

    row_fen = ''
    for c in terminated_fen:
        c_lower = c.lower()
        if c_lower in digits:
            accumulated_read_not_yet_written_pieces_in_current_row += int(c_lower)
        elif c_lower!=piece_type and c_lower != '/':
            assert c_lower in ['q', 'k', 'b', 'n', 'r']
            assert c_lower != piece_type
            accumulated_read_not_yet_written_pieces_in_current_row += 1            
        elif c_lower==piece_type:
            if accumulated_read_not_yet_written_pieces_in_current_row >= 1:
                row_fen += str(accumulated_read_not_yet_written_pieces_in_current_row)                
                accumulated_read_not_yet_written_pieces_in_current_row = 0  # reset
            row_fen += c  # don't change color compared to fen
            sub_score += unit_piece_type_score
        elif c_lower == '/':
            if accumulated_read_not_yet_written_pieces_in_current_row >= 1:
                row_fen += str(accumulated_read_not_yet_written_pieces_in_current_row)                            
                accumulated_read_not_yet_written_pieces_in_current_row = 0  # reset
            row_fen += '/'
            sub_fen += row_fen
            row_fen = ''  
        else:
            assert False
        
    return sub_fen[:-1], sub_score # [:-1] drops last added '/'

def show_filtered_out_score_and_board(fen, weights_dim8, piece_type):
    sub_fen, sub_score = get_subsolution_fen_and_subscore(fen, weights_dim8, piece_type)
    print('sub_fen: ' + sub_fen)
    print('sub_score: {:f}'.format(sub_score))
    return chess.Board(sub_fen)

Calling this function for queens, we get

In [ ]:
board = show_filtered_out_score_and_board(optimal_2020_fen, weights_dim8, 'q')
write_board_to_svg(board, 'CCC_optimal_board_filtered_q.svg')
board

Note that this is an optimal solution for the maximum (=eight) queens (and no other pieces) problem, and even one possessing some symmetry.

Filtering out the the kings, we get

In [ ]:
board = show_filtered_out_score_and_board(optimal_2020_fen, weights_dim8, 'k')
write_board_to_svg(board, 'CCC_optimal_board_filtered_k.svg')
board

This is an optimal kings only solution as well, but not the one Gurobi finds with 4 kings in 4 corners of the board. So it is adapted because of the presence of other pieces, but its score does not get lowered by it.

For the rooks, we get

In [ ]:
board = show_filtered_out_score_and_board(optimal_2020_fen, weights_dim8, 'r')
write_board_to_svg(board, 'CCC_optimal_board_filtered_r.svg')
board

The diagonal is used for 6 out of 8 rooks, where it was used by the optimal Gurobi solution for all the rooks in the rook only problem. But this solution also scores 1.

For the bishops, we get

In [ ]:
board = show_filtered_out_score_and_board(optimal_2020_fen, weights_dim8, 'b')
write_board_to_svg(board, 'CCC_optimal_board_filtered_b.svg')
board

So for the bishops we have one less piece then in an optimal solution to the bishops only problem.

For the kights

In [ ]:
board = show_filtered_out_score_and_board(optimal_2020_fen, weights_dim8, 'n')
write_board_to_svg(board, 'CCC_optimal_board_filtered_n.svg')
board

So we have only half the number of knights compared to the optimal solution to the knight only problem. They are still all on one square color though,
so that much is left of 'the pattern'.

All in all, I think it's remarkable that the score of the combined problem is so high.

$\square$

In [ ]:
# The following code will convert all the svg files to png files, 
# for example to be used in a Medium article. :)
import os
import glob
svg_list = glob.glob('*.svg')
print(svg_list)
for svg in svg_list:
    png = svg.replace('.svg', '.png')
    cmd = 'rsvg-convert -h 320 {:s} > {:s}'.format(svg, png)
    print(cmd)
    os.system(cmd)

# Conclusions


## Time to Optimal Solution

The right user cuts seem to speed up solving these problems to optimality. The extent to which can apparently depend on weights in the objective function.
Indeed when weights of pieces were all 1, adding user cuts lowered solution time from 88 to 7 seconds, but when weights were 1/n with n the maximum possible pieces of its chess piece type possible to put on a chess board without threats, the 
time to optimal solution by just adding user cuts went down from 55 to 19 seconds. Nevertheless, in both cases solver time went down, so it is always useful to consider user cuts.

## Decoupling Solutions into Chess Piece Families

By splitting the solution into sub-solutions per each chess family, we see that 
a lot of the structures of the optimal solutions per chess piece family 
are still present in the solution to the combined problem.

# Possible Speed Improvements

To minimize total solver time, it may be useful to first solve the problem with weights set to 1 and add user cuts, 
which only took 7 seconds, and then insert that solution to the problem with the 1/n weights. 
Gurobi and other solvers can be warm started with a valid solution. Doing so may reduced total solver time to solve the combined problems.


# References

[1] https://github.com/PeterSels/OnComputation/tree/master/ChessCompress

[2] https://gist.github.com/PeterSels/923d2f39fe742fce3cc13065e97c9f57

[3] https://www.gurobi.com/

Peter Sels, April 21st 2020. 

Copyright © 2020 Logically Yours BV.